# Streamlit with SQL and plotly express to make a dashboard
> An interactive dashboard for sales

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import time

comment='''Giving title to the page'''

st.set_page_config(
	layout="wide",  # Can be "centered" or "wide". In the future also "dashboard", etc.
	initial_sidebar_state="auto",  # Can be "auto", "expanded", "collapsed"
	page_title=None,  # String or None. Strings get appended with "• Streamlit".
	page_icon=None,  # String, anything supported by st.image, or None.
)

row1_1, row1_2 = st.beta_columns((3,2))
with row1_1:
    st.title('Dashboard of Company ABC')

with row1_2:
    st.text(time.strftime("%Y-%m-%d %H:%M"))


comment='''Display the data'''
host="localhost"
port = 5432
database="sqlda"
user="postgres"
password="abc123"

from sqlalchemy import create_engine
try:
    conn = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user,password,host,port,database))
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

all_customers = pd.read_sql_query('select * from customers', conn)
sales_people = pd.read_sql_query('select * from salespeople', conn)
sql_query = """select * from sales
inner join dealerships
on sales.dealership_id = dealerships.dealership_id
order by 1"""
sales_dealers = pd.read_sql_query(sql_query, conn)

latest_data = all_customers.tail(50)

st.write("Total number of Customers :", all_customers.shape[0])
st.write("Total Sales :", sales_dealers['sales_amount'].sum(), "$")

comment='''Create sidebar'''
st.sidebar.title("Filter data")

state_temp = all_customers[all_customers['state'].notna()]
city_temp = all_customers[all_customers['city'].notna()]


comment='''Sidebar dropdown mutiselection'''
state_list = st.sidebar.multiselect("Select State", state_temp['state'].unique())
city_list = st.sidebar.multiselect("Select City", city_temp['city'].unique())


if state_list!=[]:
    df_filtered = all_customers[(all_customers['state'].isin(state_list))]
    st.write(df_filtered)

if city_list!=[]:
    df_filtered = all_customers[(all_customers['city'].isin(city_list))]
    st.write(df_filtered)



def plot_value_counts_over_time(df,col,color):
    data = df[col].value_counts()
    data = pd.DataFrame(data)
    fig, ax = plt.subplots(figsize=(15, 8))
    ax.bar(data.index.values,
           data[col],
           color=color)

    fsize = 16
    plt.xlabel("Time",fontsize = fsize)
    plt.ylabel("Count of Customers",fontsize = fsize)
    plt.title("Customer Counts over Time",fontsize = fsize)

    st.write(fig)


def plot_bar_over_variable(df,col,people,color):
    data = df[col].value_counts()
    data = pd.DataFrame(data).head(20)
    fig, ax = plt.subplots(figsize=(15, 8))
    ax.bar(data.index.values,
           data[col],
           color=color)

    fsize = 16
    plt.xlabel("{}".format(col),fontsize = fsize)
    plt.ylabel("Count of {}".format(people),fontsize = fsize)
    plt.title("{} Counts by {}".format(people,col),fontsize = fsize)

    st.write(fig)

def plot_pie_over_variable(df,col,labels,people):
    data = df[col].value_counts()
    data = pd.DataFrame(data)
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.pie(data[col],labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)

    fsize = 16
    plt.xlabel("{}".format(col),fontsize = fsize)
    plt.title("{} by {}".format(people,col),fontsize = fsize)

    st.write(fig)

def pivot_count(df,col):
    data = df[col].value_counts()
    data = pd.DataFrame(data)
    return data

state_df = pivot_count(all_customers,'state')
state_df.columns = ['Customer Count']


row2_1, row2_2 = st.beta_columns((1,1))
with row2_1:
    plot_value_counts_over_time(all_customers,'date_added','purple')

with row2_2:
    fig, ax = plt.subplots(figsize=(15, 8))
    fig = px.choropleth(state_df,  # Input Pandas DataFrame
                    locations=state_df.index,  # DataFrame column with locations
                    color="Customer Count",  # DataFrame column with color values
                    hover_name="Customer Count", # DataFrame column hover info
                    locationmode = 'USA-states') # Set to plot as US States
    fig.update_layout(
        title_text = 'State Rankings by Customer Count', # Create a Title
        geo_scope='usa',  # Plot only the USA instead of globe
    )
    st.write(fig)

row3_1, row3_2 = st.beta_columns((1,1))

labels = "Female", "Male"
with row3_1:
    # plot_pie_over_variable(all_customers,'gender',labels, 'Customer')
    plot_bar_over_variable(all_customers,'city', 'Customer', 'indigo')


sales_df = pd.DataFrame(sales_dealers.groupby('state').sum()['sales_amount'])
with row3_2:
    fig, ax = plt.subplots(figsize=(15, 8))
    fig = px.choropleth(sales_df,  # Input Pandas DataFrame
                    locations=sales_df.index,  # DataFrame column with locations
                    color="sales_amount",  # DataFrame column with color values
                    hover_name="sales_amount", # DataFrame column hover info
                    locationmode = 'USA-states') # Set to plot as US States
    fig.update_layout(
        title_text = 'State Rankings by Sales Amount', # Create a Title
        geo_scope='usa',  # Plot only the USA instead of globe
    )
    st.write(fig)

row4_1, row4_2, row4_3 = st.beta_columns((1,1,1))

sql_query = """select * from salespeople
inner join dealerships
on salespeople.dealership_id = dealerships.dealership_id
where dealerships.state='CA'
order by 1"""
CA_dealers = pd.read_sql_query(sql_query, conn)

with row4_1:
    plot_pie_over_variable(all_customers,'gender',labels, 'Customer')

with row4_2:
    plot_pie_over_variable(sales_people,'gender',labels, 'Salespeople')

labels = "LA", "San Mateo"
with row4_3:
    plot_pie_over_variable(CA_dealers,'city',labels, 'California')
